# import data

In [1]:
import platform
import pandas as pd
from datetime import datetime, timedelta, time
ohlcv_data = pd.read_pickle('/Users/bokie/code/python/CMS/可转债套利/cms/stock_data_min_new.pkl')
print(ohlcv_data.keys())
for k,v in ohlcv_data.items():
    print(k,' data length: ',len(v))
print('数据起始日期',v.index[0])
print('数据结束日期',v.index[-1])
print('113642.SH数据起始日期',ohlcv_data['113642.SH'].index[0])
print('113642.SH数据结束日期',ohlcv_data['113642.SH'].index[-1])

dict_keys(['123046.SZ', '300587.SZ', '113642.SH', '603185.SH', '110061.SH', '600674.SH', '128081.SZ', '002203.SZ'])
123046.SZ  data length:  31460
300587.SZ  data length:  31460
113642.SH  data length:  13552
603185.SH  data length:  31460
110061.SH  data length:  31460
600674.SH  data length:  31460
128081.SZ  data length:  31460
002203.SZ  data length:  31460
数据起始日期 2021-12-13 09:30:00
数据结束日期 2022-06-28 15:00:00
113642.SH数据起始日期 2022-04-06 09:30:00
113642.SH数据结束日期 2022-06-28 15:00:00


# bond-stock pair

In [2]:
from lib2to3.pytree import convert

# cbond_pair = ['128081.SZ','002203.SZ']
# part = 1
# convert_price = 9.69
# stock_volume = 100/convert_price #整个回测期间不变


cbond_pair = ['123046.SZ','300587.SZ']
part = 1 
convert_price = 6.73
stock_volume = 100/convert_price  # 2021-12-13至2022-03-23
# part = 2
# convert_price = 6.74
# stock_volume = 100/convert_price  # 2022-03-24至2022-06-28


# cbond_pair = ['110061.SH','600674.SH']
# part = 1
# convert_price = 9.2
# stock_volume = 100/convert_price #整个回测期间不变


# cbond_pair = ['113642.SH','603185.SH']
# part = 1
# convert_price = 145.66
# stock_volume = 100/convert_price  # 2022-04-06至2022-06-02
# part = 2
# convert_price = 102.61
# stock_volume = 100/convert_price  # 2022-06-03至2022-06-28


In [3]:
cbond_pair_price = pd.DataFrame([])
cbond_pair_price = pd.concat([ohlcv_data[cbond_pair[0]]['close'],ohlcv_data[cbond_pair[1]]['close']],axis=1)
cbond_pair_price.columns = cbond_pair
cbond_pair_price = cbond_pair_price.dropna()
if cbond_pair[0] == '128081.SZ' or cbond_pair[0] == '110061.SH':
    cbond_pair_price = cbond_pair_price.loc['2021-12-13 15:00:00':]
elif cbond_pair[0] == '123046.SZ' and part == 1:
    cbond_pair_price = cbond_pair_price.loc['2021-12-13 15:00:00':'2022-03-23 15:00:00']
elif cbond_pair[0] == '123046.SZ' and part == 2:
    cbond_pair_price = cbond_pair_price.loc['2022-03-25 15:00:00':]
elif cbond_pair[0] == '113642.SH' and part == 1:
    cbond_pair_price = cbond_pair_price.loc['2022-04-07 15:00:00':'2022-06-02 15:00:00']
elif cbond_pair[0] == '113642.SH' and part == 2:
    cbond_pair_price = cbond_pair_price.loc['2022-06-03 15:00:00':]

# cbond_pair_price = cbond_pair_price.loc['2022-04-06 15:00:00':]
# cbond_pair_price.to_excel('cbond_pair_price.xlsx')
# cbond_pair_price.to_csv('cbond_pair_price.csv')
# cbond_pair_price

# signal calculate

先算cbond_return(日内收益率，相对于昨日收盘价)，再算cbond_position，计算cbond_position中用到的open_threshold和close_threshold分别是价格止盈价格止损信号。时间止损信号则是，每日15点时强制平仓

In [4]:
cbond_return = pd.DataFrame([])

cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
stock_preclose = cbond_pair_price.iloc[0,1] # 初始时刻的stock前收盘价

for index,row in cbond_pair_price.iterrows():
    
    cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
    cbond_return.loc[index,cbond_pair[1]] = row[cbond_pair[1]]/stock_preclose - 1

    if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
        cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
        stock_preclose = row[cbond_pair[1]]
        
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

0

In [5]:
pd.set_option('display.max_rows', None)
# pair moving trend: diff = cbond reutrn - stock return 
cbond_return['diff'] = cbond_return[cbond_pair[0]] - cbond_return[cbond_pair[1]]

len(cbond_return)

15107

# grid search

In [6]:
import warnings
warnings.filterwarnings("ignore")
grid_search_res = {}

In [7]:
open_threshold_list = [0.005,0.006,0.007,0.008,0.009,
                        0.01,0.011,0.012,0.013,0.014,0.015] # 0.005 ~ 0.01 ~ 0.015 间隔是0.001
# close_threshold_list = [0,0.001,0.002,0.003] #  0 ~ 0.001 ~ 0.002 ~ 0.003
close_threshold_list = [0.000001,0.001,0.002,0.003] #  0 ~ 0.001 ~ 0.002 ~ 0.003
for open_threshold in open_threshold_list[:1]:
    for close_threshold in close_threshold_list[:1]:
        ######################################################################################################################################################
        # open_threshold = 0.01  
        # open_threshold = 0.008
        # close_threshold = 0.001 


        open_flag = 0
        cbond_position = pd.DataFrame([])
        cbond_position.index = cbond_return.index
        for index,row in cbond_return.iterrows():
            if row['diff'] > open_threshold and open_flag==0:  # diff大于开仓阈值，并且无仓位就开仓
                open_flag = 1
                if row['diff'] > 0:  # diff 绝对值大于0.01并且diff是正数，说明bond偏高了，做空之，stock则反向开仓
                    cbond_position.loc[index,cbond_pair[0]] = -1*open_flag  #short bond, long stock
                    cbond_position.loc[index,cbond_pair[1]] = open_flag
                else:                # diff 绝对值大于0.01并且diff是负数，说明bond偏低了，做多之，stock则反向开仓
                    cbond_position.loc[index,cbond_pair[0]] = open_flag     #long bond, short stock
                    cbond_position.loc[index,cbond_pair[1]] = -1*open_flag
            
            # elif abs(row['diff']) < close_threshold and open_flag==1: # diff小于平仓阈值，并且有仓位就平仓
            elif row['diff'] < close_threshold and open_flag==1: # diff小于平仓阈值，并且有仓位就平仓
                
                open_flag = 0   # diff 绝对值小于阈值时，认为股债的偏移效应减弱，走势收敛，此时平仓
                cbond_position.loc[index,cbond_pair[0]] = open_flag
                cbond_position.loc[index,cbond_pair[1]] = open_flag
                
                
            # else: # diff介于开仓阈值和平仓阈值之间，并且diff大于0，short bond long stock,小于0则short stock long bond
            #     if row['diff'] > 0:
            #         cbond_position.loc[index,cbond_pair[0]] = -1*open_flag
            #         cbond_position.loc[index,cbond_pair[1]] = open_flag
            #     else:
            #         cbond_position.loc[index,cbond_pair[0]] = open_flag
            #         cbond_position.loc[index,cbond_pair[1]] = -1*open_flag

            if index.time()==time(15,0,0): # 每日收盘时必须平仓（时间止损信号）
                open_flag = 0 
                cbond_position.loc[index,cbond_pair[0]] = open_flag
                cbond_position.loc[index,cbond_pair[1]] = open_flag
        ######################################################################################################################################################
        # cbond_position.to_csv('cbond_position.csv')
        df_signal = pd.concat([cbond_pair_price,cbond_return,cbond_position],axis=1)
        df_signal.columns = ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
        # df_signal.to_csv('df_signal.csv')

        # 用来记录交易流水的表格,一边遍历df_signal，一边填入内容到流水表格里

        import numpy as np
        cols = ['balance','signal','bond price','stock price','bond hold','stock hold','commission','tax','reverse open']
        df_trade = pd.DataFrame(
            data = [[0,np.nan,np.nan,np.nan,0,0,0,0,np.nan]],
            columns=cols
            ) 

        balance_last = 0 # 现金账户初始余额

        bond_hold = 0 # 债券持有数量
        stock_hold = 0 # 股票持有数量

        commission_rate = 2/10000 # 佣金买卖万二
        tax_rate = 1/1000 #



        for i in range(len(df_signal)-1):
            if df_signal.values[i][-1] != df_signal.values[i+1][-1]:
                index = df_signal.index[i+1]

            # df_signal的columns
            # ['bond_price','stock_price','bond_rtn','stock_rtn','diff','bond_pos','stock_pos']
                signal = str(int(df_signal.values[i+1][-2]))+str(int(df_signal.values[i+1][-1]))
                
                bond_price = df_signal.values[i+1][0] 

                stock_price = df_signal.values[i+1][1]

                commission = (bond_price + stock_price * stock_volume) * commission_rate

                reverse_open = False #表明这笔交易，是否包含平仓后再反向开仓的操作

                if signal == '00' and stock_hold == stock_volume: #有股票持仓并且需要卖出平仓时，有印花税
                    tax = stock_price * tax_rate * stock_volume
                else:
                    tax = 0

                if signal == '00':
                    balance_last += bond_hold*bond_price
                    balance_last += stock_hold*stock_price
                    
                    bond_hold = 0
                    stock_hold = 0
                    

                if signal == '1-1':
                    '''
                    加一个检测之前持仓的模块,如果之前因为-11的信号有持仓,需要先平掉(即买债卖股票),同时再反向开仓
                    '''
                    reverse_flag = False
                    if bond_hold == -1 and stock_hold == 1*stock_volume:
                        bond_hold = 0
                        stock_hold = 0
                        balance_last -= bond_price 
                        balance_last += stock_price * stock_volume
                        commission *= 2 # 因为平掉之后再反向开仓，手续费double
                        tax = stock_price * tax_rate * stock_volume # 卖股票要收印花税
                        reverse_flag = True
                        reverse_open = True

                    bond_hold += 1
                    stock_hold += (-1 * stock_volume)
                    # stock_hold += stock_volume
                    balance_last -= bond_price
                    balance_last += stock_price * stock_volume
                    
                    if reverse_flag:
                        tax *= 2 #平仓然后反向开仓的时候 已经计算过tax了，直接double
                    else:
                        tax = stock_price * tax_rate * stock_volume # 卖股票要收印花税

                if signal == '-11':
                    '''
                    加一个检测之前持仓的模块,如果之前因为1-1的信号有持仓,需要先平掉(卖债买股票),同时再反向开仓
                    '''
                    if bond_hold == 1 and stock_hold == -1*stock_volume:
                        bond_hold = 0
                        stock_hold = 0
                        balance_last += bond_price
                        balance_last -= stock_price * stock_volume
                        commission *= 2 # 因为平掉之后再反向开仓，手续费double
                                        # 买股票无印花税
                        reverse_open = True
                    bond_hold += -1
                    stock_hold += (1 * stock_volume)
                    # stock_hold += stock_volume
                    balance_last += bond_price
                    balance_last -= stock_price * stock_volume
                

                balance = balance_last

                line = [balance,signal,bond_price,stock_price,bond_hold,stock_hold,commission,tax,reverse_open]
                # df_trade.loc[index,['balance','asset','signal','bond price','stock price','commission','tax']] = line
                # df_trade.loc[index,cols] = [0,0,np.nan,np.nan,np.nan,0,0,0,0]
                df_trade.loc[index,cols] = line

        df_trade['net value'] = df_trade['balance'] + \
                                df_trade['bond price']*df_trade['bond hold'] + \
                                df_trade['stock price']*df_trade['stock hold'] - \
                                df_trade['commission'] - \
                                df_trade['tax']       
        #把交易序列单个时间节点的扩展到整个序列
        # assert False
        df_nv = df_trade[['net value']].iloc[1:]
        df_nv['time'] = df_nv.index
        df_tmp = df_signal[['diff']]
        df_tmp['time'] = df_tmp.index
        #  merge之后再前向往后fillna
        df_nv_full =  pd.merge(df_nv,df_tmp,how='outer').sort_values(by='time').fillna(method='ffill').fillna(0)
        df_nv_full.index = df_tmp.index
        df_nv_full = df_nv_full[['net value']]
        df_trade_2 = df_trade.copy(deep=True)
        rounds = [] #只包含单个来回，每2条交易记录组成一个来回的组
        reverse_rounds = [] # 包含多个来回，信号之间包含平仓同时反向开仓的组
        for i in range(len(df_trade)-1,0,-1):
        # for i in range(len(df_trade)-1,-1,-1):
            if df_trade_2.iloc[i-1,:]['signal'] == '00':
                round = df_trade_2.iloc[i:,:]
                if len(round)>2:
                    reverse_rounds.append(round)
                else:
                    rounds.append(round)
                df_trade_2 = df_trade_2.iloc[:i]

        num_round = len(rounds)
        for i in reverse_rounds:
            num_round += len(i)-1
        # # print('single round:',num_round)
        # # print('reverse rounds: ',len(reverse_rounds))

        return_in_rounds = []
        win = 0
        lose = 0
        win_profit = 0
        lose_profit = 0
        for i in rounds:
            
            tmp_profit = (i['net value'] - i['net value'].shift(1)).values[-1]
            single_round_cost = (i['bond price'].sum() + (i['stock price']*stock_volume).sum())/4
            
            return_in_round = tmp_profit/single_round_cost
            return_in_rounds.append(return_in_round)
            
            
            if tmp_profit>0:
                win += 1
                win_profit += tmp_profit
            else:
                lose += 1
                lose_profit += tmp_profit
        # # print('win: ',win)
        # # print('lose: ',lose)
        # # print('win profit: ',win_profit)
        # # print('lose profit: ',lose_profit)
        # # print(len(return_in_rounds))

        ''' 
        先把连续反向开仓的信号拆开
        e.g.  1-1 -11 00 ---> 1-1 00 -11 00
        '''

        reverse_rounds_reconstruct = []
        for reverse_round in reverse_rounds:
            reverse_rounds0_reconstruct = []
            for i in range(0,len(reverse_round)-1): #4-2

                # tmp_cols = reverse_rounds[0].columns
                # tmp_index = reverse_rounds[0].iloc[i,:].reset_index()
                m = reverse_round.iloc[i,:].tolist()
                n = reverse_round.iloc[i+1,:].tolist()
                # reverse_rounds0_reconstruct.append(m)
                tmp_balance = m[0] + n[2]*m[4] + n[3]*m[5] 
                tmp_commission = n[6]/2
                tmp_tax = n[7]/2
                tmp_nv =  tmp_balance - tmp_commission - tmp_tax
                
                insert_line  = [tmp_balance,'00',np.nan,np.nan,0,0,tmp_commission,tmp_tax,'-',tmp_nv]
                reverse_rounds0_reconstruct.append(m)
                reverse_rounds0_reconstruct.append(insert_line)
            # reverse_rounds0_reconstruct.append(n)    
            tmp_cols = reverse_round.columns
            reverse_rounds0_reconstruct[-1] = reverse_round.iloc[-1,:].tolist()
            tmp_round = pd.DataFrame(reverse_rounds0_reconstruct[:],columns=tmp_cols)
            tmp_round
            for index,row in tmp_round.iterrows():
                if row['reverse open'] == True:
                    tmp_round.loc[index,['commission','tax']] = tmp_round.loc[index,['commission','tax']]/2
            tmp_round = tmp_round.fillna(method = 'bfill')
            reverse_rounds_reconstruct.append(tmp_round)
            tmp_round['new nv'] = tmp_round['balance'] + tmp_round['bond price']*tmp_round['bond hold'] + tmp_round['stock price']*tmp_round['stock hold'] - tmp_round['commission'] - tmp_round['tax']

        # reverse_rounds_reconstruct[0]
        reverse_rounds_detail = []
        for reverse_round in reverse_rounds_reconstruct:
            # print(len(reverse_round))
            for i in range(0,len(reverse_round)-1,2):
                df_tmp = reverse_round.iloc[i:i+2,:]
                reverse_rounds_detail.append(df_tmp)
        # print(len(reverse_rounds_detail))

        # return_in_rounds = []
        # win = 0
        # lose = 0
        # win_profit = 0
        # lose_profit = 0
        for i in reverse_rounds_detail:
            
            tmp_profit = (i['new nv'] - i['new nv'].shift(1)).values[-1]
            single_round_cost = (i['bond price'].sum() + (i['stock price']*stock_volume).sum())/4
            
            return_in_round = tmp_profit/single_round_cost
            return_in_rounds.append(return_in_round)
            
            
            if tmp_profit>0:
                win += 1
                win_profit += tmp_profit
            else:
                lose += 1
                lose_profit += tmp_profit
        # print('win: ',win)
        # print('lose: ',lose)
        # print('win profit: ',win_profit)
        # print('lose profit: ',lose_profit)
        # print(len(return_in_rounds))

        win_rate = win/(win+lose)
        win_profit_avg = win_profit/win
        lose_profit_avg = lose_profit/lose
        profit_ratio = win_profit_avg/abs(lose_profit_avg)

        total_round = win + lose 

        trade_days = len(df_nv_full.resample('D').last().dropna()) -1 
        round_per_day = total_round/trade_days
        return_per_round = sum(return_in_rounds)/len(return_in_rounds)
        # print('$'*50)
        # print('pair code:',cbond_pair)
        # print('convert price: ',convert_price)
        # print('first trade time',df_trade.index[1])
        # print('last trade time',df_trade.index[-1])
        # print('$'*50)
        # print('win: ',win)
        # print('lose: ',lose)
        # print('win profit: ',win_profit)
        # print('lose profit: ',lose_profit)
        # print('sum of win&lose profit: ',win_profit+lose_profit)
        # print('final net value:',df_trade['net value'].values[-1])
        # print('win rate: ', win_rate)
        # print('profit ratio: ',profit_ratio)
        # print('trade days: ',trade_days)
        # print('total round: ',total_round)
        # print('avg round in a day: ',round_per_day)
        # print('return_per_round: ',return_per_round)
        # print('@'*80)
        df_res = pd.DataFrame(data = [[np.nan]*14],
            columns=['cbond pair','convert price','win','lose','win profit','lose profit',
            'sum of win&lose profit','final net value','win rate',
            'profit ratio','trade days','total round','avg round in a day','return per round'])

        tmp_idx = str(df_trade.index[1]) + ' -> ' + str(df_trade.index[-1])
        tmp_line = [cbond_pair,convert_price,win,lose,win_profit,lose_profit,
                    win_profit+lose_profit,df_trade['net value'].values[-1],win_rate,
                    profit_ratio,trade_days,total_round,round_per_day,return_per_round]

        df_res.loc[tmp_idx,df_res.columns] = tmp_line
        df_res = df_res.dropna()
        df_res['open threshold'] = open_threshold
        df_res['close threshold'] = close_threshold
        
        tmp_key = str(open_threshold) +'@'+ str(close_threshold)
        grid_search_res[tmp_key] = df_res
        
        print(tmp_key)

0.005@1e-06


In [8]:
df_grid_search_res =  pd.concat(list(grid_search_res.values()),axis=0)

In [9]:
df_grid_search_res

,cbond pair,convert price,win,lose,win profit,lose profit,sum of win&lose profit,final net value,win rate,profit ratio,trade days,total round,avg round in a day,return per round,open threshold,close threshold
2021-12-14 09:30:00 -> 2022-03-23 15:00:00,"[123046.SZ, 300587.SZ]",6.73,754.0,103.0,740.152296,-183.092837,557.059459,560.650364,0.879813,0.552224,66.0,857.0,12.984848,0.002209,0.005,0.000001


In [19]:
len(df_trade)

931

In [23]:
cbond_pair_price.head(7)

,123046.SZ,300587.SZ
2021-12-13 15:00:00,350.180,20.00
2021-12-14 09:30:00,351.144,19.95
2021-12-14 09:31:00,351.370,19.96
2021-12-14 09:32:00,351.368,19.94
2021-12-14 09:33:00,351.005,19.99
2021-12-14 09:34:00,348.547,20.01
2021-12-14 09:35:00,348.900,19.97


In [25]:
cbond_return.head(15)

,123046.SZ,300587.SZ,diff
2021-12-13 15:00:00,0.000000,0.0000,0.000000
2021-12-14 09:30:00,0.002753,-0.0025,0.005253
2021-12-14 09:31:00,0.003398,-0.0020,0.005398
2021-12-14 09:32:00,0.003393,-0.0030,0.006393
2021-12-14 09:33:00,0.002356,-0.0005,0.002856
2021-12-14 09:34:00,-0.004663,0.0005,-0.005163
2021-12-14 09:35:00,-0.003655,-0.0015,-0.002155
2021-12-14 09:36:00,0.000046,0.0035,-0.003454
2021-12-14 09:37:00,0.000131,0.0040,-0.003869
2021-12-14 09:38:00,0.003141,0.0010,0.002141


In [18]:
df_trade

,balance,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,net value
0,0.000000,NaN,NaN,NaN,0.0,0.000000,0.000000,0.000000,NaN,NaN
2021-12-14 09:30:00,54.710122,-11,351.144,19.95,-1.0,14.858841,0.129516,0.000000,False,-0.129516
2021-12-14 09:34:00,-47.733061,1-1,348.547,20.01,1.0,-14.858841,0.258349,0.594651,True,2.635531
2021-12-14 09:38:00,59.878945,-11,351.280,20.02,-1.0,14.858841,0.259502,0.000000,True,5.813440
2021-12-14 09:42:00,-44.047415,1-1,349.140,20.00,1.0,-14.858841,0.258527,0.594354,True,7.062885
2021-12-14 15:00:00,5.207415,00,344.500,19.87,0.0,0.000000,0.127949,0.000000,False,5.079466
2021-12-15 09:30:00,-42.451119,1-1,344.241,19.96,1.0,-14.858841,0.128165,0.296582,False,4.782667
2021-12-15 13:22:00,55.596429,-11,341.000,19.65,-1.0,14.858841,0.253190,0.000000,True,6.319465
2021-12-15 13:26:00,-41.484765,1-1,340.814,19.67,1.0,-14.858841,0.253235,0.584547,True,6.218050
2021-12-15 13:28:00,57.365137,-11,341.104,19.63,-1.0,14.858841,0.253113,0.000000,True,7.687073


# return per round surface

In [10]:
df_return_per_round = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_return_per_round.loc[open_threshold,close_threshold] = row['return per round']
df_return_per_round

,0.000001
0.005,0.002209


In [11]:
df_return_per_round.max()

0.000001    0.002209
dtype: float64

In [12]:
df_return_per_round.min()

0.000001    0.002209
dtype: float64

In [13]:
import plotly.graph_objects as go

z_data = df_return_per_round.copy(deep=True)
fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title="return per round", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90)  # 4个位置的距离
                 )

fig.show()

# win rate surface

In [14]:
df_win_rate = pd.DataFrame([])
for index,row in df_grid_search_res.iterrows():
    open_threshold = row['open threshold']
    close_threshold = row['close threshold']
    df_win_rate.loc[open_threshold,close_threshold] = row['win rate']
df_win_rate

,0.000001
0.005,0.879813


In [15]:
df_win_rate.max()

0.000001    0.879813
dtype: float64

In [16]:
df_win_rate.min()

0.000001    0.879813
dtype: float64

In [17]:
import plotly.graph_objects as go

z_data = df_win_rate.copy(deep=True)
fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title="win rate", # 标题
                  autosize=False,  # 尺度自动缩放
                  width=700,  # 长宽
                  height=600,
                  margin=dict(l=65,r=50,b=65,t=90)  # 4个位置的距离
                 )

fig.show()